In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
#from torchvision import datasets, transforms
import pandas as pd

# 读取CSV文件
train_images = pd.read_csv('./mnistdata/train_images.csv',header = None)
test_images = pd.read_csv('./mnistdata/test_images.csv',header = None)
train_labels = pd.read_csv('./mnistdata/train_labels.csv',header = None)
test_labels = pd.read_csv('./mnistdata/test_labels.csv',header = None)

# 将像素值转换为浮点数并归一化
train_images = train_images.values.astype('float32') / 255.0
test_images = test_images.values.astype('float32') / 255.0
train_labels = train_labels.values
test_labels = test_labels.values

# 转换为PyTorch张量
train_tensors = torch.Tensor(train_images).view(train_images.shape[0],1,28,28)
train_labels = torch.LongTensor(train_labels).view(-1)
test_tensors = torch.Tensor(test_images).view(test_images.shape[0],1, 28, 28)
test_labels = torch.LongTensor(test_labels).view(-1)

print(train_tensors.size())
print(train_labels.size())

# 创建数据集和数据加载器
train_dataset = TensorDataset(train_tensors, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(test_tensors, test_labels)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
print('training samples: {}, test samples: {}'.format(len(train_dataset), len(test_dataset)))

torch.Size([60000, 1, 28, 28])
torch.Size([60000])
training samples: 60000, test samples: 10000


In [2]:
# 定义模型
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = x.view(-1, 320)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# 初始化模型和优化器
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()
num_epochs = 5
# 训练模型
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_num_train = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct_num_train += pred.eq(target.view_as(pred)).sum().item()
    running_loss += loss
    train_loss = running_loss / (batch_idx + 1)
    train_accuracy = correct_num_train / len(train_dataset)
    log_message = f'Epoch {epoch+1}/{num_epochs}, Xiao Ai Train Loss{train_loss:.4f},Xiao Ai Train Accuracy: {train_accuracy:.4f}'
    print(log_message)

C:\ProgramData\Anaconda3\envs\ai4engineeringcourse\lib\site-packages\torch\autograd\__init__.py:251: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/5, Xiao Ai Train Loss0.0002,Xiao Ai Train Accuracy: 0.7189
Epoch 2/5, Xiao Ai Train Loss0.0001,Xiao Ai Train Accuracy: 0.9382
Epoch 3/5, Xiao Ai Train Loss0.0000,Xiao Ai Train Accuracy: 0.9593
Epoch 4/5, Xiao Ai Train Loss0.0001,Xiao Ai Train Accuracy: 0.9684
Epoch 5/5, Xiao Ai Train Loss0.0000,Xiao Ai Train Accuracy: 0.9733


In [3]:
model.eval()
test_loss = 0
correct_test = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct_test += pred.eq(target.view_as(pred)).sum().item()
print('test acc={:.4f}'.format(correct_test / len(test_dataset)))

test acc=0.9793
